<a href="https://colab.research.google.com/github/DhafinRA/Apriori/blob/main/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [6]:
#membaca data excel
df = pd.read_excel('SERVIS INJEKSI MANUAL BEBEK MATIC.xlsx')

# menampilkan 5 data teratas dari data frame
df.head()

,ServTransCode,ServTransDate,Items,NameOfServ,qty,profit
0,SRV001912080003,2019-12-08,Oli Mesin Matic,SERVICE INJEKSI MANUAL BEBEK/MATIC,1,0
1,SRV001912080003,2019-12-08,Grease Cvt,SERVICE INJEKSI MANUAL BEBEK/MATIC,1,0
2,SRV001912080003,2019-12-08,Handel Variasi,SERVICE INJEKSI MANUAL BEBEK/MATIC,1,0
3,SRV001912080003,2019-12-08,Air Radiator,SERVICE INJEKSI MANUAL BEBEK/MATIC,1,0
4,SRV001912120004,2019-12-12,Oli Gardan,SERVICE INJEKSI MANUAL BEBEK/MATIC,1,0


In [7]:
# Menghilangkan spasi dalam data
df['Items'] = df['Items'].str.strip()

In [8]:
frequent_implement = apriori(df,min_support=0.07,use_colnames=True)

frequent_implement.head()

c:\Users\Ameli\miniconda3\envs\tf2\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value SRV001912080003